# Enviroment Settings

In [85]:
import arcpy
import pandas as pd
import numpy as np
import arcgis
import time
from arcpy.sa import *
from arcpy.ia import *
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [86]:
# load object for mapping
gis = GIS("pro")

# define working directory
arcpy.env.workspace = "NorthernRockies_Beetle_Fire.gdb"

# set overwrite rules for output files
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

# NAD_1983_Contiguous_USA_Albers
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(5070)

# Set the raster cell alignment environment using a keyword.
arcpy.env.cellAlignment = "ALIGN_WITH_PROCESSING_EXTENT"

# Set the raster cell size environment using a number.
arcpy.env.cellSize = 30

# list the layers in working directory
fclist = arcpy.ListFeatureClasses()
rasterList = arcpy.ListRasters()

print(fclist, '\n')
print(rasterList)

['NorthernRockies_FireHistory_2000_2020', 'NorthernRockies_Ecoregion', 'NorthernRockies_BeetleDmg_2000', 'NorthernRockies_BeetleDmg_2001', 'NorthernRockies_BeetleDmg_2002', 'NorthernRockies_BeetleDmg_2003', 'NorthernRockies_BeetleDmg_2004', 'NorthernRockies_BeetleDmg_2005', 'NorthernRockies_BeetleDmg_2006', 'NorthernRockies_BeetleDmg_2007', 'NorthernRockies_BeetleDmg_2008', 'NorthernRockies_BeetleDmg_2009', 'NorthernRockies_BeetleDmg_2010', 'NorthernRockies_BeetleDmg_2011', 'NorthernRockies_BeetleDmg_2012', 'NorthernRockies_BeetleDmg_2014', 'NorthernRockies_BeetleDmg_2015', 'NorthernRockies_BeetleDmg_2016', 'NorthernRockies_BeetleDmg_2017', 'NorthernRockies_BeetleDmg_2018', 'NorthernRockies_BeetleDmg_2019', 'NorthernRockies_BeetleDmg_2020', 'NorthernRockies_Subalpine_1_LF_Vector', 'NorthernRockies_BeetleDmg_2000_Clipped', 'NorthernRockies_BeetleDmg_2001_Clipped', 'NorthernRockies_BeetleDmg_2002_Clipped', 'NorthernRockies_BeetleDmg_2003_Clipped', 'NorthernRockies_BeetleDmg_2004_Clipped'

# Functions

In [87]:
def clippyToRasty(subalpine_layer, polygon_list):
    polygons_clipped = []
    
    for polygon in polygon_list:
        arcpy.analysis.PairwiseClip(polygon, subalpine_layer, polygon + '_Clipped')
        arcpy.management.AddField(polygon + '_Clipped', 'Value', 'SHORT')
        arcpy.management.CalculateField(polygon + '_Clipped', 'Value', '1')
        polygons_clipped.append(polygon + '_Clipped')
        print('CLIPPED {}'.format(polygon))
    
    for polygon in polygons_clipped:
        arcpy.conversion.PolygonToRaster(polygon, 'Value', polygon + '_Raster', 'CELL_CENTER', "", 30)
        print('RASTER CREATED {}'.format(polygon))
        
        
def extractValue(raster_list):
    for subalpine in raster_list:
        extract_value_1 = ExtractByAttributes(subalpine, 'Value = 1')
        extract_value_1.save(subalpine)
        print('EXTRACTED VALUE 1 from {}'.format(subalpine))

# Middle Rockies

## Geoprocessing

### Dissolve ecoregion boundary

In [8]:
arcpy.analysis.PairwiseDissolve(fclist[0], fclist[0] + '_Dissolve' )

<Result 'SouthernRockies.gdb\\SouthernRockies_Ecoregion_Dissolve'>

### Select for specific forests types Land Fire land cover layer

In [6]:
query = "Value = 3050 OR Value = 3055 OR Value = 3056"

extract = ExtractByAttributes('SouthernEcoRegion_Subalpine_LF', query)

extract.save("D:\RockiesBeetleFire\RockiesBeetleFire\SouthernRockies.gdb\SouthernEcoRegion_Subalpine_TEMP")

myRemapVal = RemapValue([[3050,1],[3055,1],[3056,1]])

outReclass = Reclassify("SouthernEcoRegion_Subalpine_TEMP", "Value", myRemapVal)

outReclass.save("D:\RockiesBeetleFire\RockiesBeetleFire\SouthernRockies.gdb\SouthernEcoRegion_Subalpine_LF_SELECTION")

arcpy.Delete_management("SouthernEcoRegion_Subalpine_TEMP")

<Result 'true'>

In [15]:
eco = ['Southern_US_Rockies_Ecoregion']

polygonToRaster(eco)

Southern_US_Rockies_Ecoregion RASTER CREATED


### Clip beetle damage to ecoregion

In [6]:
# list the layers in working directory
fclist = arcpy.ListFeatureClasses('BeetleDmg*')

fclist

['BeetleDmg_2000', 'BeetleDmg_2001', 'BeetleDmg_2002', 'BeetleDmg_2003', 'BeetleDmg_2004', 'BeetleDmg_2005', 'BeetleDmg_2006', 'BeetleDmg_2007', 'BeetleDmg_2008', 'BeetleDmg_2009', 'BeetleDmg_2010', 'BeetleDmg_2011', 'BeetleDmg_2012', 'BeetleDmg_2013', 'BeetleDmg_2014', 'BeetleDmg_2015', 'BeetleDmg_2016', 'BeetleDmg_2017', 'BeetleDmg_2018', 'BeetleDmg_2019', 'BeetleDmg_2020']

In [88]:
# years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

years = ['2013']

fclist = ['BeetleDmg_2013']

# Loop through each file and perform the processing
for fc, year in zip(fclist, years):
    
    print("Processing " + fc)
    
    arcpy.analysis.PairwiseClip(fc, r'NorthernRockies_Beetle_Fire.gdb\NorthernRockies_Ecoregion', 'NorthernRockies_BeetleDmg_{}'.format(year))

Processing BeetleDmg_2013


### Convert subalpine to vector

In [9]:
arcpy.conversion.RasterToPolygon('NorthernRockies_Subalpine_1_LF', 'NorthernRockies_Subalpine_1_LF_Vector', "NO_SIMPLIFY")

# MAKE SURE TO DELETE GRID 0's FROM ATTRIBUTE TABLE

<Result 'NorthernRockies_Beetle_Fire.gdb\\NorthernRockies_Subalpine_1_LF_Vector'>

### Clip beetle damage to subalpine and make raster

In [89]:
fclist = arcpy.ListFeatureClasses('SouthernRockies_BeetleDmg*') 

fclist = ['NorthernRockies_BeetleDmg_2013']

fclist

['NorthernRockies_BeetleDmg_2013']

In [90]:
clippyToRasty('NorthernRockies_Subalpine_1_LF_Vector', fclist) 

CLIPPED NorthernRockies_BeetleDmg_2013
RASTER CREATED NorthernRockies_BeetleDmg_2013_Clipped


### Select for fire years, clip to subalpine, and output to raster

In [14]:
# Extract fire history by years
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for year in years:
    frYear = arcpy.management.SelectLayerByAttribute('NorthernRockies_FireHistory_2000_2020', 'NEW_SELECTION', "FIRE_YEAR = '{}'".format(year))
    # Write the selected features to a new feature class
    arcpy.CopyFeatures_management(frYear, 'NorthernRockies_FireHistory_' + year)
    print('EXTRACTED FIRE YEAR {}'.format(year))

EXTRACTED FIRE YEAR 2000
EXTRACTED FIRE YEAR 2001
EXTRACTED FIRE YEAR 2002
EXTRACTED FIRE YEAR 2003
EXTRACTED FIRE YEAR 2004
EXTRACTED FIRE YEAR 2005
EXTRACTED FIRE YEAR 2006
EXTRACTED FIRE YEAR 2007
EXTRACTED FIRE YEAR 2008
EXTRACTED FIRE YEAR 2009
EXTRACTED FIRE YEAR 2010
EXTRACTED FIRE YEAR 2011
EXTRACTED FIRE YEAR 2012
EXTRACTED FIRE YEAR 2013
EXTRACTED FIRE YEAR 2014
EXTRACTED FIRE YEAR 2015
EXTRACTED FIRE YEAR 2016
EXTRACTED FIRE YEAR 2017
EXTRACTED FIRE YEAR 2018
EXTRACTED FIRE YEAR 2019
EXTRACTED FIRE YEAR 2020


In [16]:
# list the layers in working directory
fclist = arcpy.ListFeatureClasses('NorthernRockies_FireHistory_2*')[1:]

fclist

['NorthernRockies_FireHistory_2000', 'NorthernRockies_FireHistory_2001', 'NorthernRockies_FireHistory_2002', 'NorthernRockies_FireHistory_2003', 'NorthernRockies_FireHistory_2004', 'NorthernRockies_FireHistory_2005', 'NorthernRockies_FireHistory_2006', 'NorthernRockies_FireHistory_2007', 'NorthernRockies_FireHistory_2008', 'NorthernRockies_FireHistory_2009', 'NorthernRockies_FireHistory_2010', 'NorthernRockies_FireHistory_2011', 'NorthernRockies_FireHistory_2012', 'NorthernRockies_FireHistory_2013', 'NorthernRockies_FireHistory_2014', 'NorthernRockies_FireHistory_2015', 'NorthernRockies_FireHistory_2016', 'NorthernRockies_FireHistory_2017', 'NorthernRockies_FireHistory_2018', 'NorthernRockies_FireHistory_2019', 'NorthernRockies_FireHistory_2020']

In [17]:
clippyToRasty('NorthernRockies_Subalpine_1_LF_Vector', fclist) 

CLIPPED NorthernRockies_FireHistory_2000
CLIPPED NorthernRockies_FireHistory_2001
CLIPPED NorthernRockies_FireHistory_2002
CLIPPED NorthernRockies_FireHistory_2003
CLIPPED NorthernRockies_FireHistory_2004
CLIPPED NorthernRockies_FireHistory_2005
CLIPPED NorthernRockies_FireHistory_2006
CLIPPED NorthernRockies_FireHistory_2007
CLIPPED NorthernRockies_FireHistory_2008
CLIPPED NorthernRockies_FireHistory_2009
CLIPPED NorthernRockies_FireHistory_2010
CLIPPED NorthernRockies_FireHistory_2011
CLIPPED NorthernRockies_FireHistory_2012
CLIPPED NorthernRockies_FireHistory_2013
CLIPPED NorthernRockies_FireHistory_2014
CLIPPED NorthernRockies_FireHistory_2015
CLIPPED NorthernRockies_FireHistory_2016
CLIPPED NorthernRockies_FireHistory_2017
CLIPPED NorthernRockies_FireHistory_2018
CLIPPED NorthernRockies_FireHistory_2019
CLIPPED NorthernRockies_FireHistory_2020
RASTER CREATED NorthernRockies_FireHistory_2000_Clipped
RASTER CREATED NorthernRockies_FireHistory_2001_Clipped
RASTER CREATED NorthernRock

### Create burned subalpine raster dataset

In [19]:
fireHistory = arcpy.ListRasters('NorthernRockies_FireHistory*') 

fireHistory 

['NorthernRockies_FireHistory_2000_Clipped_Raster', 'NorthernRockies_FireHistory_2001_Clipped_Raster', 'NorthernRockies_FireHistory_2002_Clipped_Raster', 'NorthernRockies_FireHistory_2003_Clipped_Raster', 'NorthernRockies_FireHistory_2004_Clipped_Raster', 'NorthernRockies_FireHistory_2005_Clipped_Raster', 'NorthernRockies_FireHistory_2006_Clipped_Raster', 'NorthernRockies_FireHistory_2007_Clipped_Raster', 'NorthernRockies_FireHistory_2008_Clipped_Raster', 'NorthernRockies_FireHistory_2009_Clipped_Raster', 'NorthernRockies_FireHistory_2010_Clipped_Raster', 'NorthernRockies_FireHistory_2011_Clipped_Raster', 'NorthernRockies_FireHistory_2012_Clipped_Raster', 'NorthernRockies_FireHistory_2013_Clipped_Raster', 'NorthernRockies_FireHistory_2014_Clipped_Raster', 'NorthernRockies_FireHistory_2015_Clipped_Raster', 'NorthernRockies_FireHistory_2016_Clipped_Raster', 'NorthernRockies_FireHistory_2017_Clipped_Raster', 'NorthernRockies_FireHistory_2018_Clipped_Raster', 'NorthernRockies_FireHistory_2

In [20]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

alpineHistory = ['NorthernRockies_Subalpine_1_LF', 'NorthernRockies_SubalpineBurned_2000', 'NorthernRockies_SubalpineBurned_2001', 
                 'NorthernRockies_SubalpineBurned_2002', 'NorthernRockies_SubalpineBurned_2003', 'NorthernRockies_SubalpineBurned_2004',
                 'NorthernRockies_SubalpineBurned_2005', 'NorthernRockies_SubalpineBurned_2006', 'NorthernRockies_SubalpineBurned_2007', 
                 'NorthernRockies_SubalpineBurned_2008', 'NorthernRockies_SubalpineBurned_2009', 'NorthernRockies_SubalpineBurned_2010',
                 'NorthernRockies_SubalpineBurned_2011', 'NorthernRockies_SubalpineBurned_2012', 'NorthernRockies_SubalpineBurned_2013',
                 'NorthernRockies_SubalpineBurned_2014', 'NorthernRockies_SubalpineBurned_2015', 'NorthernRockies_SubalpineBurned_2016',
                 'NorthernRockies_SubalpineBurned_2017', 'NorthernRockies_SubalpineBurned_2018', 'NorthernRockies_SubalpineBurned_2019']
                 

for alpineRas, fireRas, year in zip(alpineHistory, fireHistory, years):
    outRas = Raster(SetNull(~IsNull(fireRas), alpineRas))
    outRas.save('NorthernRockies_Beetle_Fire.gdb\{}'.format('NorthernRockies_SubalpineBurned_' + year))
    print('RASTER CREATED {}'.format(alpineRas))  

RASTER CREATED NorthernRockies_Subalpine_1_LF
RASTER CREATED NorthernRockies_SubalpineBurned_2000
RASTER CREATED NorthernRockies_SubalpineBurned_2001
RASTER CREATED NorthernRockies_SubalpineBurned_2002
RASTER CREATED NorthernRockies_SubalpineBurned_2003
RASTER CREATED NorthernRockies_SubalpineBurned_2004
RASTER CREATED NorthernRockies_SubalpineBurned_2005
RASTER CREATED NorthernRockies_SubalpineBurned_2006
RASTER CREATED NorthernRockies_SubalpineBurned_2007
RASTER CREATED NorthernRockies_SubalpineBurned_2008
RASTER CREATED NorthernRockies_SubalpineBurned_2009
RASTER CREATED NorthernRockies_SubalpineBurned_2010
RASTER CREATED NorthernRockies_SubalpineBurned_2011
RASTER CREATED NorthernRockies_SubalpineBurned_2012
RASTER CREATED NorthernRockies_SubalpineBurned_2013
RASTER CREATED NorthernRockies_SubalpineBurned_2014
RASTER CREATED NorthernRockies_SubalpineBurned_2015
RASTER CREATED NorthernRockies_SubalpineBurned_2016
RASTER CREATED NorthernRockies_SubalpineBurned_2017
RASTER CREATED Nor

In [69]:
alpineHistory = arcpy.ListRasters('SouthernRockies_SubalpineBurned*')

alpineHistory

['SouthernRockies_SubalpineBurned_2001', 'SouthernRockies_SubalpineBurned_2002', 'SouthernRockies_SubalpineBurned_2003', 'SouthernRockies_SubalpineBurned_2004', 'SouthernRockies_SubalpineBurned_2005', 'SouthernRockies_SubalpineBurned_2006', 'SouthernRockies_SubalpineBurned_2007', 'SouthernRockies_SubalpineBurned_2008', 'SouthernRockies_SubalpineBurned_2009', 'SouthernRockies_SubalpineBurned_2010', 'SouthernRockies_SubalpineBurned_2011', 'SouthernRockies_SubalpineBurned_2012', 'SouthernRockies_SubalpineBurned_2013', 'SouthernRockies_SubalpineBurned_2014', 'SouthernRockies_SubalpineBurned_2015', 'SouthernRockies_SubalpineBurned_2016', 'SouthernRockies_SubalpineBurned_2017', 'SouthernRockies_SubalpineBurned_2018', 'SouthernRockies_SubalpineBurned_2019', 'SouthernRockies_SubalpineBurned_2020', 'SouthernRockies_SubalpineBurned_2000']

In [73]:
# extract value 1

for subalpine in alpineHistory:
    extract_value_1 = ExtractByAttributes(subalpine, 'Value = 1')
    extract_value_1.save(subalpine)
    print('EXTRACTED VALUE 1 from {}'.format(subalpine))

EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2001
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2002
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2003
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2004
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2005
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2006
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2007
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2008
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2009
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2010
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2011
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2012
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2013
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2014
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2015
EXTRACTED VALUE 1 from SouthernRockies_SubalpineBurned_2016
EXTRACTED VALUE 1 from SouthernRockies_S

### Create burned beetle raster dataset

In [91]:
# years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# list the layers in working directory
# beetleDmg_list = arcpy.ListRasters('SouthernRockies_BeetleDmg*')

years = ['2013']
beetleDmg_list = ['NorthernRockies_BeetleDmg_2013_Clipped_Raster']

In [92]:
# fireHistory = arcpy.ListRasters('SouthernRockies_FireHistory*')

fireHistory = ['NorthernRockies_FireHistory_2013_Clipped_Raster']

In [93]:
for beetleDmg, fireRas, year in zip(beetleDmg_list, fireHistory, years):
    outRas = Raster(SetNull(~IsNull(fireRas), beetleDmg))
    outRas.save('NorthernRockies_Beetle_Fire.gdb\{}'.format('NorthernRockies_BeetleDmg_Burned_' + year))  
    print('RASTER CREATED {}'.format(beetleDmg)) 

RASTER CREATED NorthernRockies_BeetleDmg_2013_Clipped_Raster


## Create Dataframe

In [94]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']


# list the layers in working directory
sub_burn_list = arcpy.ListRasters('NorthernRockies_Subalpine*')
# sub_burn_list.insert(0, sub_burn_list.pop()) # move original raster to the front of list 'SouthernRockies_Subalpine_1_LF'

beetleDmg_list = arcpy.ListRasters('NorthernRockies_BeetleDmg*')[:21]
beetleDmgBurn_list = arcpy.ListRasters('NorthernRockies_BeetleDmg_Burned*') 

print(sub_burn_list, '\n') 
print(beetleDmg_list, '\n')
print(beetleDmgBurn_list, '\n')

['NorthernRockies_Subalpine_1_LF', 'NorthernRockies_SubalpineBurned_2000', 'NorthernRockies_SubalpineBurned_2001', 'NorthernRockies_SubalpineBurned_2002', 'NorthernRockies_SubalpineBurned_2003', 'NorthernRockies_SubalpineBurned_2004', 'NorthernRockies_SubalpineBurned_2005', 'NorthernRockies_SubalpineBurned_2006', 'NorthernRockies_SubalpineBurned_2007', 'NorthernRockies_SubalpineBurned_2008', 'NorthernRockies_SubalpineBurned_2009', 'NorthernRockies_SubalpineBurned_2010', 'NorthernRockies_SubalpineBurned_2011', 'NorthernRockies_SubalpineBurned_2012', 'NorthernRockies_SubalpineBurned_2013', 'NorthernRockies_SubalpineBurned_2014', 'NorthernRockies_SubalpineBurned_2015', 'NorthernRockies_SubalpineBurned_2016', 'NorthernRockies_SubalpineBurned_2017', 'NorthernRockies_SubalpineBurned_2018', 'NorthernRockies_SubalpineBurned_2019', 'NorthernRockies_SubalpineBurned_2020'] 

['NorthernRockies_BeetleDmg_2000_Clipped_Raster', 'NorthernRockies_BeetleDmg_2001_Clipped_Raster', 'NorthernRockies_BeetleD

In [95]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

sub_burn_list = ['NorthernRockies_Subalpine_1_LF', 'NorthernRockies_SubalpineBurned_2000', 'NorthernRockies_SubalpineBurned_2001', 
                 'NorthernRockies_SubalpineBurned_2002', 'NorthernRockies_SubalpineBurned_2003', 'NorthernRockies_SubalpineBurned_2004', 
                 'NorthernRockies_SubalpineBurned_2005', 'NorthernRockies_SubalpineBurned_2006', 'NorthernRockies_SubalpineBurned_2007', 
                 'NorthernRockies_SubalpineBurned_2008', 'NorthernRockies_SubalpineBurned_2009', 'NorthernRockies_SubalpineBurned_2010', 
                 'NorthernRockies_SubalpineBurned_2011', 'NorthernRockies_SubalpineBurned_2012', 'NorthernRockies_SubalpineBurned_2013', 
                 'NorthernRockies_SubalpineBurned_2014', 'NorthernRockies_SubalpineBurned_2015', 'NorthernRockies_SubalpineBurned_2016', 
                 'NorthernRockies_SubalpineBurned_2017', 'NorthernRockies_SubalpineBurned_2018', 'NorthernRockies_SubalpineBurned_2019', 
                 'NorthernRockies_SubalpineBurned_2020'] 

beetleDmg_list = ['NorthernRockies_BeetleDmg_2000_Clipped_Raster', 'NorthernRockies_BeetleDmg_2001_Clipped_Raster', 'NorthernRockies_BeetleDmg_2002_Clipped_Raster', 
                  'NorthernRockies_BeetleDmg_2003_Clipped_Raster', 'NorthernRockies_BeetleDmg_2004_Clipped_Raster', 'NorthernRockies_BeetleDmg_2005_Clipped_Raster', 
                  'NorthernRockies_BeetleDmg_2006_Clipped_Raster', 'NorthernRockies_BeetleDmg_2007_Clipped_Raster', 'NorthernRockies_BeetleDmg_2008_Clipped_Raster', 
                  'NorthernRockies_BeetleDmg_2009_Clipped_Raster', 'NorthernRockies_BeetleDmg_2010_Clipped_Raster', 'NorthernRockies_BeetleDmg_2011_Clipped_Raster', 
                  'NorthernRockies_BeetleDmg_2012_Clipped_Raster', 'NorthernRockies_BeetleDmg_2013_Clipped_Raster', 'NorthernRockies_BeetleDmg_2014_Clipped_Raster', 
                  'NorthernRockies_BeetleDmg_2015_Clipped_Raster', 'NorthernRockies_BeetleDmg_2016_Clipped_Raster', 'NorthernRockies_BeetleDmg_2017_Clipped_Raster', 
                  'NorthernRockies_BeetleDmg_2018_Clipped_Raster', 'NorthernRockies_BeetleDmg_2019_Clipped_Raster', 'NorthernRockies_BeetleDmg_2020_Clipped_Raster']

beetleDmgBurn_list = ['NorthernRockies_BeetleDmg_Burned_2000', 'NorthernRockies_BeetleDmg_Burned_2001', 'NorthernRockies_BeetleDmg_Burned_2002', 
                      'NorthernRockies_BeetleDmg_Burned_2003', 'NorthernRockies_BeetleDmg_Burned_2004', 'NorthernRockies_BeetleDmg_Burned_2005', 
                      'NorthernRockies_BeetleDmg_Burned_2006', 'NorthernRockies_BeetleDmg_Burned_2007', 'NorthernRockies_BeetleDmg_Burned_2008', 
                      'NorthernRockies_BeetleDmg_Burned_2009', 'NorthernRockies_BeetleDmg_Burned_2010', 'NorthernRockies_BeetleDmg_Burned_2011', 
                      'NorthernRockies_BeetleDmg_Burned_2012', 'NorthernRockies_BeetleDmg_Burned_2013', 'NorthernRockies_BeetleDmg_Burned_2014', 
                      'NorthernRockies_BeetleDmg_Burned_2015', 'NorthernRockies_BeetleDmg_Burned_2016', 'NorthernRockies_BeetleDmg_Burned_2017', 
                      'NorthernRockies_BeetleDmg_Burned_2018', 'NorthernRockies_BeetleDmg_Burned_2019', 'NorthernRockies_BeetleDmg_Burned_2020']




In [96]:
# create list of cell count for each raster in list

totalSubAlpine = []

for raster in sub_burn_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            totalSubAlpine.append(row[0])

print(totalSubAlpine, '\n', len(totalSubAlpine))

[21982822.0, 21802672.0, 21522350.0, 21504129.0, 20573044.0, 20573044.0, 20529169.0, 20380801.0, 19616107.0, 19615935.0, 19598125.0, 19582052.0, 19554178.0, 19489446.0, 19463131.0, 19460834.0, 18889978.0, 18852718.0, 18338596.0, 18236972.0, 18234520.0, 18220017.0] 
 22


In [97]:
# create list of cell count for each raster in list

beetleDmg = []

for raster in beetleDmg_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            beetleDmg.append(row[0])

print(beetleDmg, '\n', len(beetleDmg))

[285749.0, 363724.0, 541532.0, 808834.0, 1062183.0, 1356314.0, 1962591.0, 2237996.0, 2563242.0, 3182505.0, 3954879.0, 4768871.0, 4889584.0, 4979670.0, 5045611.0, 5119902.0, 5173859.0, 5192561.0, 5203534.0, 5218385.0, 5239259.0] 
 21


In [98]:
# create list of cell count for each raster in list

beetleDmgBurn = []

for raster in beetleDmgBurn_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            beetleDmgBurn.append(row[0])

print(beetleDmgBurn, '\n', len(beetleDmgBurn))

[275707.0, 362614.0, 541532.0, 799848.0, 1062183.0, 1349707.0, 1959174.0, 2155923.0, 2563242.0, 3182387.0, 3949131.0, 4764944.0, 4879073.0, 4978132.0, 5044202.0, 4969982.0, 5156351.0, 5049175.0, 5191687.0, 5217388.0, 5236897.0] 
 21


In [99]:
df = pd.DataFrame({
    'Year': years,
    'Total Subalpine Forested Area (ha)': totalSubAlpine[1:],
    'Cumulative Area of Bark Beetle (ha)': beetleDmg,
    'beetleDmgBurn': beetleDmgBurn
})

# add starting totalSubAlpine count
df.loc[-1] = [np.nan, 21982822.0, np.nan, np.nan] # adding a row
df.index = df.index + 1  # shifting index
df = df.sort_index()  # sorting by index

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn
0,NaN,21982822.0,NaN,NaN
1,2000,21802672.0,285749.0,275707.0
2,2001,21522350.0,363724.0,362614.0
3,2002,21504129.0,541532.0,541532.0
4,2003,20573044.0,808834.0,799848.0
5,2004,20573044.0,1062183.0,1062183.0
6,2005,20529169.0,1356314.0,1349707.0
7,2006,20380801.0,1962591.0,1959174.0
8,2007,19616107.0,2237996.0,2155923.0
9,2008,19615935.0,2563242.0,2563242.0


In [100]:
# find actual burned barkbeetles

df['Actual Area Burned in BB Area (ha)'] = df['Cumulative Area of Bark Beetle (ha)'] - df['beetleDmgBurn']

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn,Actual Area Burned in BB Area (ha)
0,NaN,21982822.0,NaN,NaN,NaN
1,2000,21802672.0,285749.0,275707.0,10042.0
2,2001,21522350.0,363724.0,362614.0,1110.0
3,2002,21504129.0,541532.0,541532.0,0.0
4,2003,20573044.0,808834.0,799848.0,8986.0
5,2004,20573044.0,1062183.0,1062183.0,0.0
6,2005,20529169.0,1356314.0,1349707.0,6607.0
7,2006,20380801.0,1962591.0,1959174.0,3417.0
8,2007,19616107.0,2237996.0,2155923.0,82073.0
9,2008,19615935.0,2563242.0,2563242.0,0.0


In [101]:
df['Area Burned (ha)'] = df['Total Subalpine Forested Area (ha)'].diff().abs()

df['Expected Area Burned in BB Area (ha)'] = (df['Cumulative Area of Bark Beetle (ha)']/df['Total Subalpine Forested Area (ha)']) * df['Area Burned (ha)']

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn,Actual Area Burned in BB Area (ha),Area Burned (ha),Expected Area Burned in BB Area (ha)
0,NaN,21982822.0,NaN,NaN,NaN,NaN,NaN
1,2000,21802672.0,285749.0,275707.0,10042.0,180150.0,2361.072182
2,2001,21522350.0,363724.0,362614.0,1110.0,280322.0,4737.393413
3,2002,21504129.0,541532.0,541532.0,0.0,18221.0,458.853952
4,2003,20573044.0,808834.0,799848.0,8986.0,931085.0,36605.822886
5,2004,20573044.0,1062183.0,1062183.0,0.0,0.0,0.000000
6,2005,20529169.0,1356314.0,1349707.0,6607.0,43875.0,2898.718246
7,2006,20380801.0,1962591.0,1959174.0,3417.0,148368.0,14287.255024
8,2007,19616107.0,2237996.0,2155923.0,82073.0,764694.0,87243.718299
9,2008,19615935.0,2563242.0,2563242.0,0.0,172.0,22.475484


In [102]:
df = df[['Year', 'Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 
         'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']]

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),Area Burned (ha),Expected Area Burned in BB Area (ha),Actual Area Burned in BB Area (ha)
0,NaN,21982822.0,NaN,NaN,NaN,NaN
1,2000,21802672.0,285749.0,180150.0,2361.072182,10042.0
2,2001,21522350.0,363724.0,280322.0,4737.393413,1110.0
3,2002,21504129.0,541532.0,18221.0,458.853952,0.0
4,2003,20573044.0,808834.0,931085.0,36605.822886,8986.0
5,2004,20573044.0,1062183.0,0.0,0.000000,0.0
6,2005,20529169.0,1356314.0,43875.0,2898.718246,6607.0
7,2006,20380801.0,1962591.0,148368.0,14287.255024,3417.0
8,2007,19616107.0,2237996.0,764694.0,87243.718299,82073.0
9,2008,19615935.0,2563242.0,172.0,22.475484,0.0


In [103]:
# convert to hectares

hectares = lambda x : x * 900 / 10000

df[['Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']] = df[['Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']].apply(hectares)

df = df.round(2)

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),Area Burned (ha),Expected Area Burned in BB Area (ha),Actual Area Burned in BB Area (ha)
0,NaN,1978453.98,NaN,NaN,NaN,NaN
1,2000,1962240.48,25717.41,16213.50,212.50,903.78
2,2001,1937011.50,32735.16,25228.98,426.37,99.90
3,2002,1935371.61,48737.88,1639.89,41.30,0.00
4,2003,1851573.96,72795.06,83797.65,3294.52,808.74
5,2004,1851573.96,95596.47,0.00,0.00,0.00
6,2005,1847625.21,122068.26,3948.75,260.88,594.63
7,2006,1834272.09,176633.19,13353.12,1285.85,307.53
8,2007,1765449.63,201419.64,68822.46,7851.93,7386.57
9,2008,1765434.15,230691.78,15.48,2.02,0.00


In [104]:
df.to_csv(r'C:\Users\JGrov\Desktop\NorthernEcoRegion_BeetleFire_Jaredv3.csv', index=False) 

### Dissolve ecoregion boundary

In [12]:
arcpy.analysis.PairwiseDissolve(fclist[0], fclist[0] + '_Dissolve' )

<Result 'MiddleRockies.gdb\\MiddleRockies_Ecoregion_Dissolve'>

### Select for specific forests in GAP land cover and Land Fire land cover layer

In [15]:
query = "Value = 3050 OR Value = 3055 OR Value = 3056"

extract = ExtractByAttributes('MiddleEcoRegion_Subalpine_LF_30m', query)

extract.save("D:\RockiesBeetleFire\RockiesBeetleFire\MiddleRockies.gdb\MiddleEcoRegion_Subalpine_TEMP")

myRemapVal = RemapValue([[3050,1],[3055,1],[3056,1]])

outReclass = Reclassify("MiddleEcoRegion_Subalpine_TEMP", "Value", myRemapVal)

outReclass.save("D:\RockiesBeetleFire\RockiesBeetleFire\MiddleRockies.gdb\MiddleEcoRegion_Subalpine_LF_SELECTION")

arcpy.Delete_management("MiddleEcoRegion_Subalpine_TEMP")

<Result 'true'>

### Create ecoregion raster

In [8]:
eco = ['MIddleRockies_Ecoregion']

polygonToRaster(eco)

MIddleRockies_Ecoregion RASTER CREATED


### Clip beetle damage to ecoregion

In [23]:
arcpy.env.workspace = 'RockiesBeetleFire.gdb'

# list the layers in working directory
fclist = arcpy.ListFeatureClasses('BeetleDmg_2*')

fclist

['BeetleDmg_2000', 'BeetleDmg_2001', 'BeetleDmg_2002', 'BeetleDmg_2003', 'BeetleDmg_2004', 'BeetleDmg_2005', 'BeetleDmg_2006', 'BeetleDmg_2007', 'BeetleDmg_2008', 'BeetleDmg_2009', 'BeetleDmg_2010', 'BeetleDmg_2011', 'BeetleDmg_2012', 'BeetleDmg_2013', 'BeetleDmg_2014', 'BeetleDmg_2015', 'BeetleDmg_2016', 'BeetleDmg_2017', 'BeetleDmg_2018', 'BeetleDmg_2019', 'BeetleDmg_2020']

In [24]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# Loop through each file and perform the processing
for fc, year in zip(fclist, years):
    print("Processing " + fc)
    arcpy.analysis.PairwiseClip(fc, 'MiddleRockies.gdb\MiddleRockies_Ecoregion', 'MiddleRockies_BeetleDmg_{}'.format(year))

Processing BeetleDmg_2000
Processing BeetleDmg_2001
Processing BeetleDmg_2002
Processing BeetleDmg_2003
Processing BeetleDmg_2004
Processing BeetleDmg_2005
Processing BeetleDmg_2006
Processing BeetleDmg_2007
Processing BeetleDmg_2008
Processing BeetleDmg_2009
Processing BeetleDmg_2010
Processing BeetleDmg_2011
Processing BeetleDmg_2012
Processing BeetleDmg_2013
Processing BeetleDmg_2014
Processing BeetleDmg_2015
Processing BeetleDmg_2016
Processing BeetleDmg_2017
Processing BeetleDmg_2018
Processing BeetleDmg_2019
Processing BeetleDmg_2020


### Create raster datasets from beetle damage

In [10]:
# list the layers in working directory
fclist = arcpy.ListFeatureClasses('MiddleRockies_BeetleDmg*')


fclist

['MiddleRockies_BeetleDmg_2000', 'MiddleRockies_BeetleDmg_2001', 'MiddleRockies_BeetleDmg_2002', 'MiddleRockies_BeetleDmg_2003', 'MiddleRockies_BeetleDmg_2004', 'MiddleRockies_BeetleDmg_2005', 'MiddleRockies_BeetleDmg_2006', 'MiddleRockies_BeetleDmg_2007', 'MiddleRockies_BeetleDmg_2008', 'MiddleRockies_BeetleDmg_2009', 'MiddleRockies_BeetleDmg_2010', 'MiddleRockies_BeetleDmg_2011', 'MiddleRockies_BeetleDmg_2012', 'MiddleRockies_BeetleDmg_2013', 'MiddleRockies_BeetleDmg_2014', 'MiddleRockies_BeetleDmg_2015', 'MiddleRockies_BeetleDmg_2016', 'MiddleRockies_BeetleDmg_2017', 'MiddleRockies_BeetleDmg_2018', 'MiddleRockies_BeetleDmg_2019', 'MiddleRockies_BeetleDmg_2020']

In [11]:
# create BeetleDmg Raster

test = ['MiddleRockies_BeetleDmg_2020']

polygonToRaster(test)

MiddleRockies_BeetleDmg_2020 RASTER CREATED


### Create raster from middle alpine 

In [17]:
alpine_poly = ['Middle_Subalpine']
polygonToRaster(alpine_poly)

Middle_Subalpine RASTER CREATED


### Select for fire years, clip to ecoregion, and output to raster

In [18]:
# Extract fire history by years
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for year in years:
    frYear = arcpy.management.SelectLayerByAttribute('MiddleRockies_FireHistory_2000_2020', 'NEW_SELECTION', "FIRE_YEAR = {}".format(year))
    # Write the selected features to a new feature class
    arcpy.CopyFeatures_management(frYear, 'MiddleRockies_FireHistory_{}'.format(year))

In [19]:
# list the layers in working directory
fclist = arcpy.ListFeatureClasses('MiddleRockies_FireHistory_*')[1:]

fclist

['MiddleRockies_FireHistory_2000', 'MiddleRockies_FireHistory_2001', 'MiddleRockies_FireHistory_2002', 'MiddleRockies_FireHistory_2003', 'MiddleRockies_FireHistory_2004', 'MiddleRockies_FireHistory_2005', 'MiddleRockies_FireHistory_2006', 'MiddleRockies_FireHistory_2007', 'MiddleRockies_FireHistory_2008', 'MiddleRockies_FireHistory_2009', 'MiddleRockies_FireHistory_2010', 'MiddleRockies_FireHistory_2011', 'MiddleRockies_FireHistory_2012', 'MiddleRockies_FireHistory_2013', 'MiddleRockies_FireHistory_2014', 'MiddleRockies_FireHistory_2015', 'MiddleRockies_FireHistory_2016', 'MiddleRockies_FireHistory_2017', 'MiddleRockies_FireHistory_2018', 'MiddleRockies_FireHistory_2019', 'MiddleRockies_FireHistory_2020']

In [22]:
# create fireHistory rasters
polygonToRaster(fclist)

MiddleRockies_FireHistory_2000 RASTER CREATED
MiddleRockies_FireHistory_2001 RASTER CREATED
MiddleRockies_FireHistory_2002 RASTER CREATED
MiddleRockies_FireHistory_2003 RASTER CREATED
MiddleRockies_FireHistory_2004 RASTER CREATED
MiddleRockies_FireHistory_2005 RASTER CREATED
MiddleRockies_FireHistory_2006 RASTER CREATED
MiddleRockies_FireHistory_2007 RASTER CREATED
MiddleRockies_FireHistory_2008 RASTER CREATED
MiddleRockies_FireHistory_2009 RASTER CREATED
MiddleRockies_FireHistory_2010 RASTER CREATED
MiddleRockies_FireHistory_2011 RASTER CREATED
MiddleRockies_FireHistory_2012 RASTER CREATED
MiddleRockies_FireHistory_2013 RASTER CREATED
MiddleRockies_FireHistory_2014 RASTER CREATED
MiddleRockies_FireHistory_2015 RASTER CREATED
MiddleRockies_FireHistory_2016 RASTER CREATED
MiddleRockies_FireHistory_2017 RASTER CREATED
MiddleRockies_FireHistory_2018 RASTER CREATED
MiddleRockies_FireHistory_2019 RASTER CREATED
MiddleRockies_FireHistory_2020 RASTER CREATED


In [48]:
# list the layers in working directory
fclist = arcpy.ListFeatureClasses('MiddleRockies_FireHistory_2000_2020*')

fclist


['MiddleRockies_FireHistory_2000_2020']

In [49]:
# create fireHistory rasters
polygonToRaster(fclist)

MiddleRockies_FireHistory_2000_2020 RASTER CREATED


### Create burned subalpine raster dataset

In [8]:
# list the layers in working directory
fireHistory = arcpy.ListRasters('MiddleRockies_FireHistory_*')

fireHistory

[]

In [28]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

alpineHistory = ['MiddleEcoRegion_Subalpine_LF_SELECTION', 'MiddleRockies_SubalpineBurned_2000', 'MiddleRockies_SubalpineBurned_2001', 
                 'MiddleRockies_SubalpineBurned_2002', 'MiddleRockies_SubalpineBurned_2003', 'MiddleRockies_SubalpineBurned_2004',
                 'MiddleRockies_SubalpineBurned_2005', 'MiddleRockies_SubalpineBurned_2006', 'MiddleRockies_SubalpineBurned_2007', 
                 'MiddleRockies_SubalpineBurned_2008', 'MiddleRockies_SubalpineBurned_2009', 'MiddleRockies_SubalpineBurned_2010',
                 'MiddleRockies_SubalpineBurned_2011', 'MiddleRockies_SubalpineBurned_2012', 'MiddleRockies_SubalpineBurned_2013',
                 'MiddleRockies_SubalpineBurned_2014', 'MiddleRockies_SubalpineBurned_2015', 'MiddleRockies_SubalpineBurned_2016',
                 'MiddleRockies_SubalpineBurned_2017', 'MiddleRockies_SubalpineBurned_2018', 'MiddleRockies_SubalpineBurned_2019']
                 

for alpineRas, fireRas, year in zip(alpineHistory, fireHistory, years):
    outRas = Raster(SetNull(~IsNull(fireRas), alpineRas))
    outRas.save('MiddleRockies.gdb\{}'.format('MiddleRockies_SubalpineBurned_'+ year))
    print('{} RASTER CREATED'.format(alpineRas))

MiddleEcoRegion_Subalpine_LF_SELECTION RASTER CREATED
MiddleRockies_SubalpineBurned_2000 RASTER CREATED
MiddleRockies_SubalpineBurned_2001 RASTER CREATED
MiddleRockies_SubalpineBurned_2002 RASTER CREATED
MiddleRockies_SubalpineBurned_2003 RASTER CREATED
MiddleRockies_SubalpineBurned_2004 RASTER CREATED
MiddleRockies_SubalpineBurned_2005 RASTER CREATED
MiddleRockies_SubalpineBurned_2006 RASTER CREATED
MiddleRockies_SubalpineBurned_2007 RASTER CREATED
MiddleRockies_SubalpineBurned_2008 RASTER CREATED
MiddleRockies_SubalpineBurned_2009 RASTER CREATED
MiddleRockies_SubalpineBurned_2010 RASTER CREATED
MiddleRockies_SubalpineBurned_2011 RASTER CREATED
MiddleRockies_SubalpineBurned_2012 RASTER CREATED
MiddleRockies_SubalpineBurned_2013 RASTER CREATED
MiddleRockies_SubalpineBurned_2014 RASTER CREATED
MiddleRockies_SubalpineBurned_2015 RASTER CREATED
MiddleRockies_SubalpineBurned_2016 RASTER CREATED
MiddleRockies_SubalpineBurned_2017 RASTER CREATED
MiddleRockies_SubalpineBurned_2018 RASTER CREA

In [36]:
alpineburn = arcpy.ListRasters('MiddleRockies_Subalpine*')

alpineburn

['MIddleRockies_Subalpine_1_Start', 'MiddleRockies_SubalpineBurned_2000', 'MiddleRockies_SubalpineBurned_2001', 'MiddleRockies_SubalpineBurned_2002', 'MiddleRockies_SubalpineBurned_2003', 'MiddleRockies_SubalpineBurned_2004', 'MiddleRockies_SubalpineBurned_2005', 'MiddleRockies_SubalpineBurned_2006', 'MiddleRockies_SubalpineBurned_2007', 'MiddleRockies_SubalpineBurned_2008', 'MiddleRockies_SubalpineBurned_2009', 'MiddleRockies_SubalpineBurned_2010', 'MiddleRockies_SubalpineBurned_2011', 'MiddleRockies_SubalpineBurned_2012', 'MiddleRockies_SubalpineBurned_2013', 'MiddleRockies_SubalpineBurned_2014', 'MiddleRockies_SubalpineBurned_2015', 'MiddleRockies_SubalpineBurned_2016', 'MiddleRockies_SubalpineBurned_2017', 'MiddleRockies_SubalpineBurned_2018', 'MiddleRockies_SubalpineBurned_2019', 'MiddleRockies_SubalpineBurned_2020']

In [37]:
for raster in alpineburn:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            print(row[0])

40456034.0
38995582.0
38783419.0
38705974.0
38107285.0
38102015.0
37785305.0
37205053.0
34953741.0
34707596.0
34620367.0
34561398.0
34391753.0
33353211.0
33170974.0
33161939.0
32755610.0
32331932.0
31502367.0
31290092.0
31264112.0
31128824.0


### Create burned beetle raster dataset

In [38]:
beetleDmg_list = arcpy.ListRasters('MiddleRockies_BeetleDmg*')

beetleDmg_list

['MiddleRockies_BeetleDmg_2000_Raster', 'MiddleRockies_BeetleDmg_2001_Raster', 'MiddleRockies_BeetleDmg_2002_Raster', 'MiddleRockies_BeetleDmg_2003_Raster', 'MiddleRockies_BeetleDmg_2004_Raster', 'MiddleRockies_BeetleDmg_2005_Raster', 'MiddleRockies_BeetleDmg_2006_Raster', 'MiddleRockies_BeetleDmg_2007_Raster', 'MiddleRockies_BeetleDmg_2008_Raster', 'MiddleRockies_BeetleDmg_2009_Raster', 'MiddleRockies_BeetleDmg_2010_Raster', 'MiddleRockies_BeetleDmg_2011_Raster', 'MiddleRockies_BeetleDmg_2012_Raster', 'MiddleRockies_BeetleDmg_2013_Raster', 'MiddleRockies_BeetleDmg_2014_Raster', 'MiddleRockies_BeetleDmg_2015_Raster', 'MiddleRockies_BeetleDmg_2016_Raster', 'MiddleRockies_BeetleDmg_2017_Raster', 'MiddleRockies_BeetleDmg_2018_Raster', 'MiddleRockies_BeetleDmg_2019_Raster', 'MiddleRockies_BeetleDmg_2020_Raster']

In [39]:
fireHistory = arcpy.ListRasters('MiddleRockies_FireHistory_*')

fireHistory

['MiddleRockies_FireHistory_2000_Raster', 'MiddleRockies_FireHistory_2001_Raster', 'MiddleRockies_FireHistory_2002_Raster', 'MiddleRockies_FireHistory_2003_Raster', 'MiddleRockies_FireHistory_2004_Raster', 'MiddleRockies_FireHistory_2005_Raster', 'MiddleRockies_FireHistory_2006_Raster', 'MiddleRockies_FireHistory_2007_Raster', 'MiddleRockies_FireHistory_2008_Raster', 'MiddleRockies_FireHistory_2009_Raster', 'MiddleRockies_FireHistory_2010_Raster', 'MiddleRockies_FireHistory_2011_Raster', 'MiddleRockies_FireHistory_2012_Raster', 'MiddleRockies_FireHistory_2013_Raster', 'MiddleRockies_FireHistory_2014_Raster', 'MiddleRockies_FireHistory_2015_Raster', 'MiddleRockies_FireHistory_2016_Raster', 'MiddleRockies_FireHistory_2017_Raster', 'MiddleRockies_FireHistory_2018_Raster', 'MiddleRockies_FireHistory_2019_Raster', 'MiddleRockies_FireHistory_2020_Raster']

In [40]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for beetleDmg, fireRas, year in zip(beetleDmg_list, fireHistory, years):
    outRas = Raster(SetNull(~IsNull(fireRas), beetleDmg))
    outRas.save('MiddleRockies.gdb\{}'.format('MiddleRockies_BeetleDmg_Burned_' + year))
    print('{} RASTER CREATED'.format(beetleDmg))

MiddleRockies_BeetleDmg_2000_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2001_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2002_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2003_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2004_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2005_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2006_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2007_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2008_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2009_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2010_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2011_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2012_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2013_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2014_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2015_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2016_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2017_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2018_Raster RASTER CREATED
MiddleRockies_BeetleDmg_2019_Ra

In [72]:
beetleDmg_Burned = arcpy.ListRasters('MiddleRockies_BeetleDmg_Burned_*')

beetleDmg_Burned

['MiddleRockies_BeetleDmg_Burned_2000', 'MiddleRockies_BeetleDmg_Burned_2001', 'MiddleRockies_BeetleDmg_Burned_2002', 'MiddleRockies_BeetleDmg_Burned_2003', 'MiddleRockies_BeetleDmg_Burned_2004', 'MiddleRockies_BeetleDmg_Burned_2005', 'MiddleRockies_BeetleDmg_Burned_2006', 'MiddleRockies_BeetleDmg_Burned_2007', 'MiddleRockies_BeetleDmg_Burned_2008', 'MiddleRockies_BeetleDmg_Burned_2009', 'MiddleRockies_BeetleDmg_Burned_2010', 'MiddleRockies_BeetleDmg_Burned_2011', 'MiddleRockies_BeetleDmg_Burned_2012', 'MiddleRockies_BeetleDmg_Burned_2013', 'MiddleRockies_BeetleDmg_Burned_2014', 'MiddleRockies_BeetleDmg_Burned_2015', 'MiddleRockies_BeetleDmg_Burned_2016', 'MiddleRockies_BeetleDmg_Burned_2017', 'MiddleRockies_BeetleDmg_Burned_2018', 'MiddleRockies_BeetleDmg_Burned_2019', 'MiddleRockies_BeetleDmg_Burned_2020']

In [73]:
for raster in beetleDmg_Burned:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            print(row[0])

27189.0
87290.0
223349.0
478589.0
564813.0
635664.0
782780.0
889994.0
925562.0
1204946.0
1461270.0
1709877.0
1913092.0
2003745.0
2112675.0
2197187.0
2239621.0
2305130.0
2319684.0
2313190.0
2325458.0


In [42]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']


# list the layers in working directory
sub_burn_list = arcpy.ListRasters('MiddleRockies_Subalpine*')
# sub_burn_list.insert(0, sub_burn_list.pop()) # move original raster to the front of list 'MIddleRockies_Subalpine_1_Start'

beetleDmg_list = arcpy.ListRasters('MiddleRockies_BeetleDmg*')[:21]
beetleDmgBurn_list = arcpy.ListRasters('MiddleRockies_BeetleDmg_Burned*')

print(sub_burn_list, '\n')
print(beetleDmg_list, '\n')
print(beetleDmgBurn_list, '\n')

['MIddleRockies_Subalpine_1_Start', 'MiddleRockies_SubalpineBurned_2000', 'MiddleRockies_SubalpineBurned_2001', 'MiddleRockies_SubalpineBurned_2002', 'MiddleRockies_SubalpineBurned_2003', 'MiddleRockies_SubalpineBurned_2004', 'MiddleRockies_SubalpineBurned_2005', 'MiddleRockies_SubalpineBurned_2006', 'MiddleRockies_SubalpineBurned_2007', 'MiddleRockies_SubalpineBurned_2008', 'MiddleRockies_SubalpineBurned_2009', 'MiddleRockies_SubalpineBurned_2010', 'MiddleRockies_SubalpineBurned_2011', 'MiddleRockies_SubalpineBurned_2012', 'MiddleRockies_SubalpineBurned_2013', 'MiddleRockies_SubalpineBurned_2014', 'MiddleRockies_SubalpineBurned_2015', 'MiddleRockies_SubalpineBurned_2016', 'MiddleRockies_SubalpineBurned_2017', 'MiddleRockies_SubalpineBurned_2018', 'MiddleRockies_SubalpineBurned_2019', 'MiddleRockies_SubalpineBurned_2020'] 

['MiddleRockies_BeetleDmg_2000_Raster', 'MiddleRockies_BeetleDmg_2001_Raster', 'MiddleRockies_BeetleDmg_2002_Raster', 'MiddleRockies_BeetleDmg_2003_Raster', 'Middle

In [43]:
# create list of cell count for each raster in list

totalSubAlpine = []

for raster in sub_burn_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            totalSubAlpine.append(row[0])

print(totalSubAlpine, '\n', len(totalSubAlpine))

[40456034.0, 38995582.0, 38783419.0, 38705974.0, 38107285.0, 38102015.0, 37785305.0, 37205053.0, 34953741.0, 34707596.0, 34620367.0, 34561398.0, 34391753.0, 33353211.0, 33170974.0, 33161939.0, 32755610.0, 32331932.0, 31502367.0, 31290092.0, 31264112.0, 31128824.0] 
 22


In [44]:
# create list of cell count for each raster in list

beetleDmg = []

for raster in beetleDmg_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            beetleDmg.append(row[0])

print(beetleDmg, '\n', len(beetleDmg))

[573519.0, 907294.0, 1648426.0, 3157725.0, 4640880.0, 7710054.0, 10606862.0, 12863680.0, 16309518.0, 23999195.0, 36425339.0, 41656852.0, 43912071.0, 43912071.0, 46877259.0, 48080072.0, 48424290.0, 48571299.0, 48628061.0, 48769760.0, 48867087.0] 
 21


In [45]:
# create list of cell count for each raster in list

beetleDmgBurn = []

for raster in beetleDmgBurn_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            beetleDmgBurn.append(row[0])

print(beetleDmgBurn, '\n', len(beetleDmgBurn))

[538233.0, 905817.0, 1647396.0, 3085139.0, 4640789.0, 7571244.0, 10401051.0, 12549615.0, 16247198.0, 23955140.0, 36369552.0, 41331624.0, 42168806.0, 43563069.0, 46859953.0, 47773044.0, 48085865.0, 47714988.0, 48322498.0, 48697996.0, 48687979.0] 
 21


In [46]:
df = pd.DataFrame({
    'Year': years,
    'Total Subalpine Forested Area (ha)': totalSubAlpine[1:],
    'Cumulative Area of Bark Beetle (ha)': beetleDmg,
    'beetleDmgBurn': beetleDmgBurn
})

# add starting totalSubAlpine count
df.loc[-1] = [np.nan, 40456034.0, np.nan, np.nan] # adding a row
df.index = df.index + 1  # shifting index
df = df.sort_index()  # sorting by index

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn
0,NaN,40456034.0,NaN,NaN
1,2000,38995582.0,573519.0,538233.0
2,2001,38783419.0,907294.0,905817.0
3,2002,38705974.0,1648426.0,1647396.0
4,2003,38107285.0,3157725.0,3085139.0
5,2004,38102015.0,4640880.0,4640789.0
6,2005,37785305.0,7710054.0,7571244.0
7,2006,37205053.0,10606862.0,10401051.0
8,2007,34953741.0,12863680.0,12549615.0
9,2008,34707596.0,16309518.0,16247198.0


In [47]:
# find actual burned barkbeetles

df['Actual Area Burned in BB Area (ha)'] = df['Cumulative Area of Bark Beetle (ha)'] - df['beetleDmgBurn']

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn,Actual Area Burned in BB Area (ha)
0,NaN,40456034.0,NaN,NaN,NaN
1,2000,38995582.0,573519.0,538233.0,35286.0
2,2001,38783419.0,907294.0,905817.0,1477.0
3,2002,38705974.0,1648426.0,1647396.0,1030.0
4,2003,38107285.0,3157725.0,3085139.0,72586.0
5,2004,38102015.0,4640880.0,4640789.0,91.0
6,2005,37785305.0,7710054.0,7571244.0,138810.0
7,2006,37205053.0,10606862.0,10401051.0,205811.0
8,2007,34953741.0,12863680.0,12549615.0,314065.0
9,2008,34707596.0,16309518.0,16247198.0,62320.0


In [48]:
df['Area Burned (ha)'] = df['Total Subalpine Forested Area (ha)'].diff().abs()

df['Expected Area Burned in BB Area (ha)'] = (df['Cumulative Area of Bark Beetle (ha)']/df['Total Subalpine Forested Area (ha)']) * df['Area Burned (ha)']

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn,Actual Area Burned in BB Area (ha),Area Burned (ha),Expected Area Burned in BB Area (ha)
0,NaN,40456034.0,NaN,NaN,NaN,NaN,NaN
1,2000,38995582.0,573519.0,538233.0,35286.0,1460452.0,2.147928e+04
2,2001,38783419.0,907294.0,905817.0,1477.0,212163.0,4.963312e+03
3,2002,38705974.0,1648426.0,1647396.0,1030.0,77445.0,3.298260e+03
4,2003,38107285.0,3157725.0,3085139.0,72586.0,598689.0,4.960981e+04
5,2004,38102015.0,4640880.0,4640789.0,91.0,5270.0,6.418935e+02
6,2005,37785305.0,7710054.0,7571244.0,138810.0,316710.0,6.462436e+04
7,2006,37205053.0,10606862.0,10401051.0,205811.0,580252.0,1.654252e+05
8,2007,34953741.0,12863680.0,12549615.0,314065.0,2251312.0,8.285281e+05
9,2008,34707596.0,16309518.0,16247198.0,62320.0,246145.0,1.156665e+05


In [49]:
df = df[['Year', 'Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 
         'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']]

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),Area Burned (ha),Expected Area Burned in BB Area (ha),Actual Area Burned in BB Area (ha)
0,NaN,40456034.0,NaN,NaN,NaN,NaN
1,2000,38995582.0,573519.0,1460452.0,2.147928e+04,35286.0
2,2001,38783419.0,907294.0,212163.0,4.963312e+03,1477.0
3,2002,38705974.0,1648426.0,77445.0,3.298260e+03,1030.0
4,2003,38107285.0,3157725.0,598689.0,4.960981e+04,72586.0
5,2004,38102015.0,4640880.0,5270.0,6.418935e+02,91.0
6,2005,37785305.0,7710054.0,316710.0,6.462436e+04,138810.0
7,2006,37205053.0,10606862.0,580252.0,1.654252e+05,205811.0
8,2007,34953741.0,12863680.0,2251312.0,8.285281e+05,314065.0
9,2008,34707596.0,16309518.0,246145.0,1.156665e+05,62320.0


In [50]:
# convert to hectares

hectares = lambda x : x * 900 / 10000

df[['Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']] = df[['Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']].apply(hectares)

df = df.round(2)

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),Area Burned (ha),Expected Area Burned in BB Area (ha),Actual Area Burned in BB Area (ha)
0,NaN,3641043.06,NaN,NaN,NaN,NaN
1,2000,3509602.38,51616.71,131440.68,1933.14,3175.74
2,2001,3490507.71,81656.46,19094.67,446.70,132.93
3,2002,3483537.66,148358.34,6970.05,296.84,92.70
4,2003,3429655.65,284195.25,53882.01,4464.88,6532.74
5,2004,3429181.35,417679.20,474.30,57.77,8.19
6,2005,3400677.45,693904.86,28503.90,5816.19,12492.90
7,2006,3348454.77,954617.58,52222.68,14888.27,18522.99
8,2007,3145836.69,1157731.20,202618.08,74567.53,28265.85
9,2008,3123683.64,1467856.62,22153.05,10409.99,5608.80


In [52]:
df.to_csv(r'C:\Users\JGrov\Desktop\MiddleEcoRegion_BeetleFire_jared.csv')

In [4]:
# define working directory
arcpy.env.workspace = "NorthernRockies.gdb"

# set overwrite rules for output files
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

# NAD_1983_Contiguous_USA_Albers
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(5070)

# Set the raster cell alignment environment using a keyword.
arcpy.env.cellAlignment = "ALIGN_WITH_PROCESSING_EXTENT"

# Set the raster cell size environment using a number.
arcpy.env.cellSize = 30

fclist = arcpy.ListFeatureClasses()

fclist

['NorthernRockies_Ecoregion', 'NIFC_NorthernEcoRegion_2020', 'MTBS_NorthernEcoRegion_2000_2019', 'NorthernRockies_FireHistory_2000_2020', 'NorthernRockies_BeetleDmg_2000', 'NorthernRockies_BeetleDmg_2001', 'NorthernRockies_BeetleDmg_2002', 'NorthernRockies_BeetleDmg_2003', 'NorthernRockies_BeetleDmg_2004', 'NorthernRockies_BeetleDmg_2005', 'NorthernRockies_BeetleDmg_2006', 'NorthernRockies_BeetleDmg_2007', 'NorthernRockies_BeetleDmg_2008', 'NorthernRockies_BeetleDmg_2009', 'NorthernRockies_BeetleDmg_2010', 'NorthernRockies_BeetleDmg_2011', 'NorthernRockies_BeetleDmg_2012', 'NorthernRockies_BeetleDmg_2013', 'NorthernRockies_BeetleDmg_2014', 'NorthernRockies_BeetleDmg_2015', 'NorthernRockies_BeetleDmg_2016', 'NorthernRockies_BeetleDmg_2017', 'NorthernRockies_BeetleDmg_2018', 'NorthernRockies_BeetleDmg_2019', 'NorthernRockies_BeetleDmg_2020', 'NorthernRockies_FireHistory_2000', 'NorthernRockies_FireHistory_2001', 'NorthernRockies_FireHistory_2002', 'NorthernRockies_FireHistory_2003', 'Nor

### Dissolve ecoregion boundary

In [16]:
arcpy.analysis.PairwiseDissolve(fclist[0], fclist[0] + '_Dissolve')

<Result 'NorthernRockies.gdb\\NorthernRockies_Ecoregion_Dissolve'>

### Select for specific forests in GAP land cover and Land Fire land cover layer

In [17]:
query = "Value = 3050 OR Value = 3055 OR Value = 3056"

extract = ExtractByAttributes('NorthernEcoRegion_Subalpine_LF_30m', query)

extract.save(r"D:\RockiesBeetleFire\RockiesBeetleFire\NorthernRockies.gdb\NorthernEcoRegion_Subalpine_TEMP")

myRemapVal = RemapValue([[3050,1],[3055,1],[3056,1]])

outReclass = Reclassify("NorthernEcoRegion_Subalpine_TEMP", "Value", myRemapVal)

outReclass.save(r"D:\RockiesBeetleFire\RockiesBeetleFire\NorthernRockies.gdb\NorthernEcoRegion_Subalpine_LF_SELECTION")

arcpy.Delete_management("NorthernEcoRegion_Subalpine_TEMP")

<Result 'true'>

### Create EcoRegion Raster

In [101]:
poly = ['Northern_Rockies_Ecoregion']

polygonToRaster(poly)

Northern_Rockies_Ecoregion RASTER CREATED


### Clip Beetle Damage to Northern Rockies

In [15]:
arcpy.env.workspace = "RockiesBeetleFire.gdb"
# list the layers in working directory
fclist = arcpy.ListFeatureClasses('BeetleDmg*')

fclist

['BeetleDmg_2000', 'BeetleDmg_2001', 'BeetleDmg_2002', 'BeetleDmg_2003', 'BeetleDmg_2004', 'BeetleDmg_2005', 'BeetleDmg_2006', 'BeetleDmg_2007', 'BeetleDmg_2008', 'BeetleDmg_2009', 'BeetleDmg_2010', 'BeetleDmg_2011', 'BeetleDmg_2012', 'BeetleDmg_2013', 'BeetleDmg_2014', 'BeetleDmg_2015', 'BeetleDmg_2016', 'BeetleDmg_2017', 'BeetleDmg_2018', 'BeetleDmg_2019', 'BeetleDmg_2020']

In [16]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# Loop through each file and perform the processing
for fc, year in zip(fclist, years):
    print("processing " + fc)
    arcpy.analysis.PairwiseClip(fc, r'NorthernRockies.gdb\NorthernRockies_Ecoregion', 'NorthernRockies_BeetleDmg_{}'.format(year))

processing BeetleDmg_2000
processing BeetleDmg_2001
processing BeetleDmg_2002
processing BeetleDmg_2003
processing BeetleDmg_2004
processing BeetleDmg_2005
processing BeetleDmg_2006
processing BeetleDmg_2007
processing BeetleDmg_2008
processing BeetleDmg_2009
processing BeetleDmg_2010
processing BeetleDmg_2011
processing BeetleDmg_2012
processing BeetleDmg_2013
processing BeetleDmg_2014
processing BeetleDmg_2015
processing BeetleDmg_2016
processing BeetleDmg_2017
processing BeetleDmg_2018
processing BeetleDmg_2019
processing BeetleDmg_2020


### Create raster datasets from beetle damage

In [19]:
# list the layers in working directory
fclist = arcpy.ListFeatureClasses('NorthernRockies_BeetleDmg*')

fclist

['NorthernRockies_BeetleDmg_2000', 'NorthernRockies_BeetleDmg_2001', 'NorthernRockies_BeetleDmg_2002', 'NorthernRockies_BeetleDmg_2003', 'NorthernRockies_BeetleDmg_2004', 'NorthernRockies_BeetleDmg_2005', 'NorthernRockies_BeetleDmg_2006', 'NorthernRockies_BeetleDmg_2007', 'NorthernRockies_BeetleDmg_2008', 'NorthernRockies_BeetleDmg_2009', 'NorthernRockies_BeetleDmg_2010', 'NorthernRockies_BeetleDmg_2011', 'NorthernRockies_BeetleDmg_2012', 'NorthernRockies_BeetleDmg_2013', 'NorthernRockies_BeetleDmg_2014', 'NorthernRockies_BeetleDmg_2015', 'NorthernRockies_BeetleDmg_2016', 'NorthernRockies_BeetleDmg_2017', 'NorthernRockies_BeetleDmg_2018', 'NorthernRockies_BeetleDmg_2019', 'NorthernRockies_BeetleDmg_2020']

In [20]:
# create BeetleDmg Raster

polygonToRaster(fclist)

NorthernRockies_BeetleDmg_2000 RASTER CREATED
NorthernRockies_BeetleDmg_2001 RASTER CREATED
NorthernRockies_BeetleDmg_2002 RASTER CREATED
NorthernRockies_BeetleDmg_2003 RASTER CREATED
NorthernRockies_BeetleDmg_2004 RASTER CREATED
NorthernRockies_BeetleDmg_2005 RASTER CREATED
NorthernRockies_BeetleDmg_2006 RASTER CREATED
NorthernRockies_BeetleDmg_2007 RASTER CREATED
NorthernRockies_BeetleDmg_2008 RASTER CREATED
NorthernRockies_BeetleDmg_2009 RASTER CREATED
NorthernRockies_BeetleDmg_2010 RASTER CREATED
NorthernRockies_BeetleDmg_2011 RASTER CREATED
NorthernRockies_BeetleDmg_2012 RASTER CREATED
NorthernRockies_BeetleDmg_2013 RASTER CREATED
NorthernRockies_BeetleDmg_2014 RASTER CREATED
NorthernRockies_BeetleDmg_2015 RASTER CREATED
NorthernRockies_BeetleDmg_2016 RASTER CREATED
NorthernRockies_BeetleDmg_2017 RASTER CREATED
NorthernRockies_BeetleDmg_2018 RASTER CREATED
NorthernRockies_BeetleDmg_2019 RASTER CREATED
NorthernRockies_BeetleDmg_2020 RASTER CREATED


### Create northern alpine raster

In [109]:
alpine_poly = ['Northern_Subalpine']

polygonToRaster(alpine_poly)

Northern_Subalpine RASTER CREATED


### Select for FireHistory, Clip to Northern ecoregion, and create rasters

In [113]:
arcpy.analysis.PairwiseClip('NorthernRockies_FireHistory_2000_2020', 'Northern_Rockies_Ecoregion', 'NorthernRockies_FireHistory_2000_2020_clipped')

<Result 'NorthernRockies.gdb\\NorthernRockies_FireHistory_2000_2020_clipped'>

In [21]:
# Extract fire history by years
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for year in years:
    frYear = arcpy.management.SelectLayerByAttribute('NorthernRockies_FireHistory_2000_2020', 'NEW_SELECTION', "FIRE_YEAR = '{}'".format(year))
    # Write the selected features to a new feature class
    arcpy.CopyFeatures_management(frYear, 'NorthernRockies_FireHistory_{}'.format(year))

In [30]:
# list the layers in working directory
fclist = arcpy.ListFeatureClasses('NorthernRockies_FireHistory_2*')

fclist

['NorthernRockies_FireHistory_2000_2020', 'NorthernRockies_FireHistory_2000', 'NorthernRockies_FireHistory_2001', 'NorthernRockies_FireHistory_2002', 'NorthernRockies_FireHistory_2003', 'NorthernRockies_FireHistory_2005', 'NorthernRockies_FireHistory_2006', 'NorthernRockies_FireHistory_2007', 'NorthernRockies_FireHistory_2008', 'NorthernRockies_FireHistory_2009', 'NorthernRockies_FireHistory_2010', 'NorthernRockies_FireHistory_2011', 'NorthernRockies_FireHistory_2012', 'NorthernRockies_FireHistory_2013', 'NorthernRockies_FireHistory_2014', 'NorthernRockies_FireHistory_2015', 'NorthernRockies_FireHistory_2016', 'NorthernRockies_FireHistory_2017', 'NorthernRockies_FireHistory_2018', 'NorthernRockies_FireHistory_2019', 'NorthernRockies_FireHistory_2020', 'NorthernRockies_FireHistory_2004']

In [31]:
# create fireHistory rasters
polygonToRaster(fclist)

NorthernRockies_FireHistory_2000_2020 RASTER CREATED
NorthernRockies_FireHistory_2000 RASTER CREATED
NorthernRockies_FireHistory_2001 RASTER CREATED
NorthernRockies_FireHistory_2002 RASTER CREATED
NorthernRockies_FireHistory_2003 RASTER CREATED
NorthernRockies_FireHistory_2005 RASTER CREATED
NorthernRockies_FireHistory_2006 RASTER CREATED
NorthernRockies_FireHistory_2007 RASTER CREATED
NorthernRockies_FireHistory_2008 RASTER CREATED
NorthernRockies_FireHistory_2009 RASTER CREATED
NorthernRockies_FireHistory_2010 RASTER CREATED
NorthernRockies_FireHistory_2011 RASTER CREATED
NorthernRockies_FireHistory_2012 RASTER CREATED
NorthernRockies_FireHistory_2013 RASTER CREATED
NorthernRockies_FireHistory_2014 RASTER CREATED
NorthernRockies_FireHistory_2015 RASTER CREATED
NorthernRockies_FireHistory_2016 RASTER CREATED
NorthernRockies_FireHistory_2017 RASTER CREATED
NorthernRockies_FireHistory_2018 RASTER CREATED
NorthernRockies_FireHistory_2019 RASTER CREATED
NorthernRockies_FireHistory_2020 RA

In [5]:
beetleDmg_list = arcpy.ListRasters('NorthernRockies_BeetleDmg*')

beetleDmg_list

['NorthernRockies_BeetleDmg_2000_Raster', 'NorthernRockies_BeetleDmg_2001_Raster', 'NorthernRockies_BeetleDmg_2002_Raster', 'NorthernRockies_BeetleDmg_2003_Raster', 'NorthernRockies_BeetleDmg_2004_Raster', 'NorthernRockies_BeetleDmg_2005_Raster', 'NorthernRockies_BeetleDmg_2006_Raster', 'NorthernRockies_BeetleDmg_2007_Raster', 'NorthernRockies_BeetleDmg_2008_Raster', 'NorthernRockies_BeetleDmg_2009_Raster', 'NorthernRockies_BeetleDmg_2010_Raster', 'NorthernRockies_BeetleDmg_2011_Raster', 'NorthernRockies_BeetleDmg_2012_Raster', 'NorthernRockies_BeetleDmg_2013_Raster', 'NorthernRockies_BeetleDmg_2014_Raster', 'NorthernRockies_BeetleDmg_2015_Raster', 'NorthernRockies_BeetleDmg_2016_Raster', 'NorthernRockies_BeetleDmg_2017_Raster', 'NorthernRockies_BeetleDmg_2018_Raster', 'NorthernRockies_BeetleDmg_2019_Raster', 'NorthernRockies_BeetleDmg_2020_Raster']

In [6]:
fireHistory = arcpy.ListRasters('NorthernRockies_FireHistory_*')

fireHistory = ['NorthernRockies_FireHistory_2000_Raster', 'NorthernRockies_FireHistory_2001_Raster', 'NorthernRockies_FireHistory_2002_Raster', 
               'NorthernRockies_FireHistory_2003_Raster', 'NorthernRockies_FireHistory_2004_Raster', 'NorthernRockies_FireHistory_2005_Raster', 
               'NorthernRockies_FireHistory_2006_Raster', 'NorthernRockies_FireHistory_2007_Raster', 'NorthernRockies_FireHistory_2008_Raster', 
               'NorthernRockies_FireHistory_2009_Raster', 'NorthernRockies_FireHistory_2010_Raster', 'NorthernRockies_FireHistory_2011_Raster', 
               'NorthernRockies_FireHistory_2012_Raster', 'NorthernRockies_FireHistory_2013_Raster', 'NorthernRockies_FireHistory_2014_Raster', 
               'NorthernRockies_FireHistory_2015_Raster', 'NorthernRockies_FireHistory_2016_Raster', 'NorthernRockies_FireHistory_2017_Raster', 
               'NorthernRockies_FireHistory_2018_Raster', 'NorthernRockies_FireHistory_2019_Raster', 'NorthernRockies_FireHistory_2020_Raster']

fireHistory

['NorthernRockies_FireHistory_2000_Raster', 'NorthernRockies_FireHistory_2001_Raster', 'NorthernRockies_FireHistory_2002_Raster', 'NorthernRockies_FireHistory_2003_Raster', 'NorthernRockies_FireHistory_2004_Raster', 'NorthernRockies_FireHistory_2005_Raster', 'NorthernRockies_FireHistory_2006_Raster', 'NorthernRockies_FireHistory_2007_Raster', 'NorthernRockies_FireHistory_2008_Raster', 'NorthernRockies_FireHistory_2009_Raster', 'NorthernRockies_FireHistory_2010_Raster', 'NorthernRockies_FireHistory_2011_Raster', 'NorthernRockies_FireHistory_2012_Raster', 'NorthernRockies_FireHistory_2013_Raster', 'NorthernRockies_FireHistory_2014_Raster', 'NorthernRockies_FireHistory_2015_Raster', 'NorthernRockies_FireHistory_2016_Raster', 'NorthernRockies_FireHistory_2017_Raster', 'NorthernRockies_FireHistory_2018_Raster', 'NorthernRockies_FireHistory_2019_Raster', 'NorthernRockies_FireHistory_2020_Raster']

In [7]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for beetleDmg, fireRas, year in zip(beetleDmg_list, fireHistory, years):
    outRas = Raster(SetNull(~IsNull(fireRas), beetleDmg))
    outRas.save('NorthernRockies.gdb\{}'.format('NorthernRockies_BeetleDmg_Burned_' + year))
    print('{} RASTER CREATED'.format(beetleDmg))

NorthernRockies_BeetleDmg_2000_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2001_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2002_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2003_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2004_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2005_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2006_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2007_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2008_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2009_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2010_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2011_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2012_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2013_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2014_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2015_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2016_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2017_Raster RASTER CREATED
NorthernRockies_BeetleDmg_2018_Raster RASTER C

In [9]:
# # list the layers in working directory
# fireHistory = arcpy.ListRasters('NorthernRockies_FireHistory*')

fireHistory = ['NorthernRockies_FireHistory_2000_Raster', 'NorthernRockies_FireHistory_2001_Raster', 'NorthernRockies_FireHistory_2002_Raster', 
               'NorthernRockies_FireHistory_2003_Raster', 'NorthernRockies_FireHistory_2004_Raster', 'NorthernRockies_FireHistory_2005_Raster', 
               'NorthernRockies_FireHistory_2006_Raster', 'NorthernRockies_FireHistory_2007_Raster', 'NorthernRockies_FireHistory_2008_Raster', 
               'NorthernRockies_FireHistory_2009_Raster', 'NorthernRockies_FireHistory_2010_Raster', 'NorthernRockies_FireHistory_2011_Raster', 
               'NorthernRockies_FireHistory_2012_Raster', 'NorthernRockies_FireHistory_2013_Raster', 'NorthernRockies_FireHistory_2014_Raster', 
               'NorthernRockies_FireHistory_2015_Raster', 'NorthernRockies_FireHistory_2016_Raster', 'NorthernRockies_FireHistory_2017_Raster', 
               'NorthernRockies_FireHistory_2018_Raster', 'NorthernRockies_FireHistory_2019_Raster', 'NorthernRockies_FireHistory_2020_Raster']

fireHistory

['NorthernRockies_FireHistory_2000_Raster', 'NorthernRockies_FireHistory_2001_Raster', 'NorthernRockies_FireHistory_2002_Raster', 'NorthernRockies_FireHistory_2003_Raster', 'NorthernRockies_FireHistory_2004_Raster', 'NorthernRockies_FireHistory_2005_Raster', 'NorthernRockies_FireHistory_2006_Raster', 'NorthernRockies_FireHistory_2007_Raster', 'NorthernRockies_FireHistory_2008_Raster', 'NorthernRockies_FireHistory_2009_Raster', 'NorthernRockies_FireHistory_2010_Raster', 'NorthernRockies_FireHistory_2011_Raster', 'NorthernRockies_FireHistory_2012_Raster', 'NorthernRockies_FireHistory_2013_Raster', 'NorthernRockies_FireHistory_2014_Raster', 'NorthernRockies_FireHistory_2015_Raster', 'NorthernRockies_FireHistory_2016_Raster', 'NorthernRockies_FireHistory_2017_Raster', 'NorthernRockies_FireHistory_2018_Raster', 'NorthernRockies_FireHistory_2019_Raster', 'NorthernRockies_FireHistory_2020_Raster']

In [10]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

alpineHistory = ['NorthernEcoRegion_Subalpine_LF_SELECTION', 'Northern_Subalpine_Burned_2000', 'Northern_Subalpine_Burned_2001', 
                 'Northern_Subalpine_Burned_2002', 'Northern_Subalpine_Burned_2003', 'Northern_Subalpine_Burned_2004',
                 'Northern_Subalpine_Burned_2005', 'Northern_Subalpine_Burned_2006', 'Northern_Subalpine_Burned_2007', 
                 'Northern_Subalpine_Burned_2008', 'Northern_Subalpine_Burned_2009', 'Northern_Subalpine_Burned_2010',
                 'Northern_Subalpine_Burned_2011', 'Northern_Subalpine_Burned_2012', 'Northern_Subalpine_Burned_2013',
                 'Northern_Subalpine_Burned_2014', 'Northern_Subalpine_Burned_2015', 'Northern_Subalpine_Burned_2016',
                 'Northern_Subalpine_Burned_2017', 'Northern_Subalpine_Burned_2018', 'Northern_Subalpine_Burned_2019']
                 

for alpineRas, fireRas, year in zip(alpineHistory, fireHistory, years):
    outRas = Raster(SetNull(~IsNull(fireRas), alpineRas))
    outRas.save('NorthernRockies.gdb\{}'.format('Northern_Subalpine_Burned_'+ year))
    print('{} RASTER CREATED'.format(alpineRas))

NorthernEcoRegion_Subalpine_LF_SELECTION RASTER CREATED
Northern_Subalpine_Burned_2000 RASTER CREATED
Northern_Subalpine_Burned_2001 RASTER CREATED
Northern_Subalpine_Burned_2002 RASTER CREATED
Northern_Subalpine_Burned_2003 RASTER CREATED
Northern_Subalpine_Burned_2004 RASTER CREATED
Northern_Subalpine_Burned_2005 RASTER CREATED
Northern_Subalpine_Burned_2006 RASTER CREATED
Northern_Subalpine_Burned_2007 RASTER CREATED
Northern_Subalpine_Burned_2008 RASTER CREATED
Northern_Subalpine_Burned_2009 RASTER CREATED
Northern_Subalpine_Burned_2010 RASTER CREATED
Northern_Subalpine_Burned_2011 RASTER CREATED
Northern_Subalpine_Burned_2012 RASTER CREATED
Northern_Subalpine_Burned_2013 RASTER CREATED
Northern_Subalpine_Burned_2014 RASTER CREATED
Northern_Subalpine_Burned_2015 RASTER CREATED
Northern_Subalpine_Burned_2016 RASTER CREATED
Northern_Subalpine_Burned_2017 RASTER CREATED
Northern_Subalpine_Burned_2018 RASTER CREATED
Northern_Subalpine_Burned_2019 RASTER CREATED


In [19]:
years = ['2000', '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']


# list the layers in working directory
sub_burn_list = arcpy.ListRasters('Northern_Subalpine_Burned*')
# sub_burn_list.insert(0, sub_burn_list.pop()) # move original raster to the front of list 'MIddleRockies_Subalpine_1_Start'
sub_burn_list.append('NorthernEcoRegion_Subalpine_LF_SELECTION')
sub_burn_list.insert(0, sub_burn_list.pop())

beetleDmg_list = arcpy.ListRasters('NorthernRockies_BeetleDmg*')[:21]
beetleDmgBurn_list = arcpy.ListRasters('NorthernRockies_BeetleDmg_Burned*')

print(sub_burn_list, '\n')
print(beetleDmg_list, '\n')
print(beetleDmgBurn_list, '\n')

['NorthernEcoRegion_Subalpine_LF_SELECTION', 'Northern_Subalpine_Burned_2000', 'Northern_Subalpine_Burned_2001', 'Northern_Subalpine_Burned_2002', 'Northern_Subalpine_Burned_2003', 'Northern_Subalpine_Burned_2004', 'Northern_Subalpine_Burned_2005', 'Northern_Subalpine_Burned_2006', 'Northern_Subalpine_Burned_2007', 'Northern_Subalpine_Burned_2008', 'Northern_Subalpine_Burned_2009', 'Northern_Subalpine_Burned_2010', 'Northern_Subalpine_Burned_2011', 'Northern_Subalpine_Burned_2012', 'Northern_Subalpine_Burned_2013', 'Northern_Subalpine_Burned_2014', 'Northern_Subalpine_Burned_2015', 'Northern_Subalpine_Burned_2016', 'Northern_Subalpine_Burned_2017', 'Northern_Subalpine_Burned_2018', 'Northern_Subalpine_Burned_2019', 'Northern_Subalpine_Burned_2020'] 

['NorthernRockies_BeetleDmg_2000_Raster', 'NorthernRockies_BeetleDmg_2001_Raster', 'NorthernRockies_BeetleDmg_2002_Raster', 'NorthernRockies_BeetleDmg_2003_Raster', 'NorthernRockies_BeetleDmg_2004_Raster', 'NorthernRockies_BeetleDmg_2005_R

In [20]:
# create list of cell count for each raster in list

totalSubAlpine = []

for raster in sub_burn_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            totalSubAlpine.append(row[0])

print(totalSubAlpine, '\n', len(totalSubAlpine))

[21982822.0, 21802930.0, 21522629.0, 21504460.0, 20572938.0, 20572938.0, 20529090.0, 20380734.0, 19616166.0, 19615993.0, 19598155.0, 19582106.0, 19554122.0, 19489326.0, 19463073.0, 19460764.0, 18890070.0, 18852838.0, 18338579.0, 18237048.0, 18234597.0, 18220144.0] 
 22


In [21]:
# create list of cell count for each raster in list

beetleDmg = []

for raster in beetleDmg_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            beetleDmg.append(row[0])

print(beetleDmg, '\n', len(beetleDmg))

[718146.0, 881086.0, 1247613.0, 1897649.0, 2514105.0, 3123764.0, 4356962.0, 4995046.0, 5564070.0, 6570909.0, 8196022.0, 9650492.0, 9984136.0, 9984136.0, 10339746.0, 10490479.0, 10606457.0, 10648750.0, 10685365.0, 10803692.0, 11013511.0] 
 21


In [22]:
# create list of cell count for each raster in list

beetleDmgBurn = []

for raster in beetleDmgBurn_list:
    fields = ['Count']
    with arcpy.da.SearchCursor(raster, fields) as cursor:
        for row in cursor:
            beetleDmgBurn.append(row[0])

print(beetleDmgBurn, '\n', len(beetleDmgBurn))

[695342.0, 879561.0, 1247613.0, 1880361.0, 2514105.0, 3106978.0, 4350911.0, 4847159.0, 5564060.0, 6570766.0, 8179683.0, 9643335.0, 9957530.0, 9971037.0, 10335229.0, 10255896.0, 10570473.0, 10351913.0, 10662596.0, 10800999.0, 11010161.0] 
 21


In [23]:
df = pd.DataFrame({
    'Year': years,
    'Total Subalpine Forested Area (ha)': totalSubAlpine[1:],
    'Cumulative Area of Bark Beetle (ha)': beetleDmg,
    'beetleDmgBurn': beetleDmgBurn
})

# add starting totalSubAlpine count
df.loc[-1] = [np.nan, 21982822.0, np.nan, np.nan] # adding a row
df.index = df.index + 1  # shifting index
df = df.sort_index()  # sorting by index

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn
0,NaN,21982822.0,NaN,NaN
1,2000,21802930.0,718146.0,695342.0
2,2001,21522629.0,881086.0,879561.0
3,2002,21504460.0,1247613.0,1247613.0
4,2003,20572938.0,1897649.0,1880361.0
5,2004,20572938.0,2514105.0,2514105.0
6,2005,20529090.0,3123764.0,3106978.0
7,2006,20380734.0,4356962.0,4350911.0
8,2007,19616166.0,4995046.0,4847159.0
9,2008,19615993.0,5564070.0,5564060.0


In [24]:
# find actual burned barkbeetles

df['Actual Area Burned in BB Area (ha)'] = df['Cumulative Area of Bark Beetle (ha)'] - df['beetleDmgBurn']

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn,Actual Area Burned in BB Area (ha)
0,NaN,21982822.0,NaN,NaN,NaN
1,2000,21802930.0,718146.0,695342.0,22804.0
2,2001,21522629.0,881086.0,879561.0,1525.0
3,2002,21504460.0,1247613.0,1247613.0,0.0
4,2003,20572938.0,1897649.0,1880361.0,17288.0
5,2004,20572938.0,2514105.0,2514105.0,0.0
6,2005,20529090.0,3123764.0,3106978.0,16786.0
7,2006,20380734.0,4356962.0,4350911.0,6051.0
8,2007,19616166.0,4995046.0,4847159.0,147887.0
9,2008,19615993.0,5564070.0,5564060.0,10.0


In [25]:
df['Area Burned (ha)'] = df['Total Subalpine Forested Area (ha)'].diff().abs()

df['Expected Area Burned in BB Area (ha)'] = (df['Cumulative Area of Bark Beetle (ha)']/df['Total Subalpine Forested Area (ha)']) * df['Area Burned (ha)']

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),beetleDmgBurn,Actual Area Burned in BB Area (ha),Area Burned (ha),Expected Area Burned in BB Area (ha)
0,NaN,21982822.0,NaN,NaN,NaN,NaN,NaN
1,2000,21802930.0,718146.0,695342.0,22804.0,179892.0,5925.291703
2,2001,21522629.0,881086.0,879561.0,1525.0,280301.0,11474.866146
3,2002,21504460.0,1247613.0,1247613.0,0.0,18169.0,1054.101363
4,2003,20572938.0,1897649.0,1880361.0,17288.0,931522.0,85923.643564
5,2004,20572938.0,2514105.0,2514105.0,0.0,0.0,0.000000
6,2005,20529090.0,3123764.0,3106978.0,16786.0,43848.0,6672.034848
7,2006,20380734.0,4356962.0,4350911.0,6051.0,148356.0,31715.317734
8,2007,19616166.0,4995046.0,4847159.0,147887.0,764568.0,194689.029963
9,2008,19615993.0,5564070.0,5564060.0,10.0,173.0,49.071393


In [26]:
df = df[['Year', 'Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 
         'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']]

df

,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),Area Burned (ha),Expected Area Burned in BB Area (ha),Actual Area Burned in BB Area (ha)
0,NaN,21982822.0,NaN,NaN,NaN,NaN
1,2000,21802930.0,718146.0,179892.0,5925.291703,22804.0
2,2001,21522629.0,881086.0,280301.0,11474.866146,1525.0
3,2002,21504460.0,1247613.0,18169.0,1054.101363,0.0
4,2003,20572938.0,1897649.0,931522.0,85923.643564,17288.0
5,2004,20572938.0,2514105.0,0.0,0.000000,0.0
6,2005,20529090.0,3123764.0,43848.0,6672.034848,16786.0
7,2006,20380734.0,4356962.0,148356.0,31715.317734,6051.0
8,2007,19616166.0,4995046.0,764568.0,194689.029963,147887.0
9,2008,19615993.0,5564070.0,173.0,49.071393,10.0


In [27]:
# convert to hectares

hectares = lambda x : x * 900 / 10000

df[['Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']] = df[['Total Subalpine Forested Area (ha)', 'Cumulative Area of Bark Beetle (ha)', 'Area Burned (ha)', 'Expected Area Burned in BB Area (ha)', 'Actual Area Burned in BB Area (ha)']].apply(hectares)

df = df.round(2)

df

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,Total Subalpine Forested Area (ha),Cumulative Area of Bark Beetle (ha),Area Burned (ha),Expected Area Burned in BB Area (ha),Actual Area Burned in BB Area (ha)
0,NaN,1978453.98,NaN,NaN,NaN,NaN
1,2000,1962263.70,64633.14,16190.28,533.28,2052.36
2,2001,1937036.61,79297.74,25227.09,1032.74,137.25
3,2002,1935401.40,112285.17,1635.21,94.87,0.00
4,2003,1851564.42,170788.41,83836.98,7733.13,1555.92
5,2004,1851564.42,226269.45,0.00,0.00,0.00
6,2005,1847618.10,281138.76,3946.32,600.48,1510.74
7,2006,1834266.06,392126.58,13352.04,2854.38,544.59
8,2007,1765454.94,449554.14,68811.12,17522.01,13309.83
9,2008,1765439.37,500766.30,15.57,4.42,0.90


In [28]:
df.to_csv(r'C:\Users\JGrov\Desktop\NorthernEcoRegion_BeetleFire_jared.csv')